In [ ]:
# 找到peer-validated的pair,写入label_count_round_3

import json

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json"
labels = ["entailment", "neutral", "contradiction"]

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        peer_validated_count = {}

        for label in labels:
            explanations = data.get(label, [])
            count = 0

            for entry in explanations:
                if not isinstance(entry, dict):
                    continue
                ak = entry.get("annotator")
                peer_yes_votes = 0

                for j in entry.get("judgments", []):
                    if not isinstance(j, dict):
                        continue
                    if j.get("annotator") != ak and j.get("makes_sense") is True:
                        peer_yes_votes += 1

                if peer_yes_votes >= 2:
                    count += 1

            peer_validated_count[label] = count if count > 0 else None

        data["label_count_round_3"] = peer_validated_count
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 新文件已生成，每条数据均包含完整的 peer_validated 字段（三类标签，值为整数或 null）。")


In [ ]:
# aggregated --> label_set_round_3

import json

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer.json"
labels = ["entailment", "neutral", "contradiction"]

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        label_count_round_3 = data.get("label_count_round_3", {})

        label_set_round_3 = []
        for label in labels:
            count = label_count_round_3.get(label)
            if count is not None and count >= 1:
                label_set_round_3.append(label)

        # 新增到数据里
        data["label_set_round_3"] = label_set_round_3

        # 保存
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 新文件已生成，每条数据都包含了 label_set_round_3。")


In [2]:
## label_set_round_1 - label_set_round_3 = error_peer
import json

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json"

all_passed = True
error_records = []  # 用来收集不符合要求的 id

# 先整体检查
with open(input_path, "r", encoding="utf-8") as infile:
    for line in infile:
        data = json.loads(line)

        label_set_round_1 = set(data.get("label_set_round_1", []))
        label_set_round_2 = set(data.get("label_set_round_2", []))
        error_labels = set(data.get("error_labels", []))

        if label_set_round_1 - label_set_round_2 != error_labels:
            all_passed = False
            error_records.append(data.get("id", "unknown_id"))

# 检查结果
if all_passed:
    print("✅ 全部正确！现在开始写入新的文件...")

    with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
        for line in infile:
            data = json.loads(line)

            label_set_round_1 = set(data.get("label_set_round_1", []))
            label_set_round_3 = set(data.get("label_set_round_3", []))

            error_peer = list(label_set_round_1 - label_set_round_3)
            data["error_peer"] = error_peer

            outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

    print(f"✅ 已生成新文件，路径为：{output_path}")

else:
    print("❌ 有不符合的记录！以下是出错的样本 ID：")
    for eid in error_records:
        print(eid)



✅ 全部正确！现在开始写入新的文件...
✅ 已生成新文件，路径为：/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json


In [9]:
## random noise

import json
import random

def inject_multiple_noises(input_path, output_path, seed, label_choices=["entailment", "neutral", "contradiction"]):
    random.seed(seed) 
    with open(input_path, 'r', encoding='utf-8') as fin, open(output_path, 'w', encoding='utf-8') as fout:
        for line in fin:
            data = json.loads(line.strip())
            error_labels = data.get("error_peer", [])

            if error_labels:
                noise_labels = [random.choice(label_choices) for _ in error_labels]
                data["random_noise_peer"] = noise_labels

            fout.write(json.dumps(data) + "\n")

    print(f"Injected noise into {output_path}")

inject_multiple_noises("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json", "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_44.json", seed=44)


Injected noise into /Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_44.json


In [33]:
# new count 

import json
from collections import Counter

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_44.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_count_44.json"


labels = ["entailment", "neutral", "contradiction"]

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)

        label_set_round_3 = set(data.get("label_set_round_3", []))
        noise_peer = data.get("random_noise_peer", [])

        # 统计 error_peer 中每个标签出现次数
        noise_counter = Counter(noise_peer)

        # 构建 random_noise_labels_count
        random_noise_labels_count = {}
        for label in labels:
            count = 0

            # 如果 label 在 label_set_round_3 出现过，加1
            if label in label_set_round_3:
                count += 1

            count += noise_counter.get(label, 0)

            # 如果最终 count 是0，就设为 null，否则 float(count)
            random_noise_labels_count[label] = float(count) if count > 0 else None

        # 只新增 random_noise_labels_count，不动 label_set_round_3
        data["random_noise_labels_count"] = random_noise_labels_count

        # 保存新的数据
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 完成：", output_path)



✅ 完成： /Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_count_44.json


{"id": "10229n", "context": "The governing statute provides that a committee consisting of the Comptroller General, the Speaker of the House and President Pro Tempore of the Senate, the Majority and Minority leaders, and the Chairmen and Ranking Minority Members of the Senate Governmental Affairs and House Government Reform Committees recommend an individual to the President for appointment.", "statement": "The process is long and will be reformed in the coming years.", "entailment": [], "neutral": [{"annotator": 0, "id": "531-neutral-1", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "The context makes no mention about how long the process takes and whether it will be reformed.", "self_corrected": false}, {"annotator": 1, "id": "531-neutral-2", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "While the context could suggest that the process takes long because so many individuals are involved, it does not say anything about reform.", "self_corrected": false}, {"annotator": 2, "id": "531-neutral-3", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "\"Reform\" is not mentioned in the context.", "self_corrected": false}, {"annotator": 3, "id": "531-neutral-4", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "The time duration of the process is not mentioned", "self_corrected": false}], "contradiction": [], "idk": [], "label_count_round_1": {"contradiction": null, "entailment": null, "neutral": 4.0}, "label_count_round_2": {"contradiction": null, "entailment": null, "neutral": 4.0}, "label_set_round_1": ["neutral"], "label_set_round_2": ["neutral"], "error_labels": [], "has_ambiguity": false, "chaosnli_labels": {"c": 18, "n": 79, "e": 3}, "labels": [0.0, 1.0, 0.0]}


In [ ]:
## 生成distribution

import json

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/500/LLM_exp+LLM_dist+LLM_validate/llama_70b/llama33_70b_explanation_with_validation_count.jsonl"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/500/LLM_exp+LLM_dist+LLM_validate/llama_70b/llama33_70b_explanation_with_validation_dist.jsonl"


def convert_label_count_to_dist(label_count_dict):

    c = label_count_dict.get("contradiction") or 0
    e = label_count_dict.get("entailment") or 0
    n = label_count_dict.get("neutral") or 0

    total = c + e + n
    if total == 0:
        return [0.0, 0.0, 0.0]  # 避免除0
    return [e / total, n / total, c / total]  # 顺序为 [e, n, c]

with open(input_path, "r", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    for line in infile:
        item = json.loads(line)

        ## change here "random_noise_labels_count"/"distribution_noise_label_count"
        label_counts = item.get("label_set_round_2")
        if label_counts:
            item["label"] = convert_label_count_to_dist(label_counts)

        json.dump(item, outfile)
        outfile.write("\n")

print(f"处理完成，已写入新文件：{output_path}")


AttributeError: 'list' object has no attribute 'get'

In [39]:
## clean

import json
from pathlib import Path
from tqdm import tqdm

def normalize_label_dist(chaos_dict):
    # 标签顺序固定：["entailment", "neutral", "contradiction"]
    label_order = ["entailment", "neutral", "contradiction"]
    values = [(chaos_dict.get(k) or 0.0) for k in label_order]  # 将 None 转换为 0.0
    total = sum(values)
    if total == 0:
        return [0.0] * 3
    return [v / total for v in values]


def process_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, desc=f"Processing {input_path}"):
            raw = json.loads(line)
            out = {
                "uid": raw.get("id", raw.get("uid")),
                "premise": raw.get("context") or raw["example"]["premise"],
                "hypothesis": raw.get("statement") or raw["example"]["hypothesis"],
                "label": raw.get("label")
            }
            fout.write(json.dumps(out, ensure_ascii=False) + "\n")

# 示例调用方式
if __name__ == "__main__":
    input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_dist_44.json"
    output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_cleaned_44.json"
    process_file(input_path, output_path)


Processing /Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_random_noise/varierr_peer_random_dist_44.json: 500it [00:00, 8749.40it/s]


# distributional

In [42]:
## distributional

import json
import random

def inject_noise_with_distribution(input_file, output_file, seed):
    label_pool = ['entailment', 'neutral', 'contradiction']
    label_weights = [87, 69, 82]
    random.seed(seed)

    with open(input_file, 'r', encoding='utf-8') as fin, \
         open(output_file, 'w', encoding='utf-8') as fout:
        
        for line in fin:
            data = json.loads(line.strip())

            error_count = len(data.get("error_peer", []))
            if error_count > 0:
                noise_labels = random.choices(label_pool, weights=label_weights, k=error_count)
                data["distribution_noise_label"] = noise_labels 
            fout.write(json.dumps(data, ensure_ascii=False) + "\n")

inject_noise_with_distribution("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json", "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_44.json", seed=44)


In [45]:
# new count 

import json
from collections import Counter

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_44.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_count_44.json"


labels = ["entailment", "neutral", "contradiction"]

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)

        label_set_round_3 = set(data.get("label_set_round_3", []))
        noise_peer = data.get("distribution_noise_label", [])

        # 统计 error_peer 中每个标签出现次数
        noise_counter = Counter(noise_peer)

        # 构建 random_noise_labels_count
        dist_noise_labels_count = {}
        for label in labels:
            count = 0

            # 如果 label 在 label_set_round_3 出现过，加1
            if label in label_set_round_3:
                count += 1

            # 再加上在 error_peer 里出现的次数
            count += noise_counter.get(label, 0)

            # 如果最终 count 是0，就设为 null，否则 float(count)
            dist_noise_labels_count[label] = float(count) if count > 0 else None

        # 只新增 random_noise_labels_count，不动 label_set_round_3
        data["dist_noise_labels_count"] = dist_noise_labels_count

        # 保存新的数据
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 完成：", output_path)



✅ 完成： /Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_count_44.json


In [49]:
## 生成distribution

import json

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_count_44.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_dist_44.json"


def convert_label_count_to_dist(label_count_dict):

    c = label_count_dict.get("contradiction") or 0
    e = label_count_dict.get("entailment") or 0
    n = label_count_dict.get("neutral") or 0

    total = c + e + n
    if total == 0:
        return [0.0, 0.0, 0.0]  # 避免除0
    return [e / total, n / total, c / total]  # 顺序为 [e, n, c]

with open(input_path, "r", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    for line in infile:
        item = json.loads(line)

        ## change here "random_noise_labels_count"/"dist_noise_labels_count"
        label_counts = item.get("dist_noise_labels_count")
        if label_counts:
            item["label"] = convert_label_count_to_dist(label_counts)

        json.dump(item, outfile)
        outfile.write("\n")

print(f"处理完成，已写入新文件：{output_path}")


处理完成，已写入新文件：/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_dist_44.json


In [52]:
## clean

import json
from pathlib import Path
from tqdm import tqdm

def normalize_label_dist(chaos_dict):
    # 标签顺序固定：["entailment", "neutral", "contradiction"]
    label_order = ["entailment", "neutral", "contradiction"]
    values = [(chaos_dict.get(k) or 0.0) for k in label_order]  # 将 None 转换为 0.0
    total = sum(values)
    if total == 0:
        return [0.0] * 3
    return [v / total for v in values]


def process_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, desc=f"Processing {input_path}"):
            raw = json.loads(line)
            out = {
                "uid": raw.get("id", raw.get("uid")),
                "premise": raw.get("context") or raw["example"]["premise"],
                "hypothesis": raw.get("statement") or raw["example"]["hypothesis"],
                "label": raw.get("label")
            }
            fout.write(json.dumps(out, ensure_ascii=False) + "\n")

# 示例调用方式
if __name__ == "__main__":
    input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_dist_44.json"
    output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_cleaned_44.json"
    process_file(input_path, output_path)


Processing /Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_peer_dist_noise/varierr_peer_dist_dist_44.json: 500it [00:00, 31114.09it/s]


## peer-validated without noise

In [ ]:
import json
from collections import Counter

input_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr_with_peer_validated.json"
output_path = "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/repeated/no_error/varierr_peer_no_error_dist.json"

labels_order = ["entailment", "neutral", "contradiction"]

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)

        label_count_round = data.get("label_count_round_3", [])
  
        cleaned_counts = {label: (0.0 if label_count_round.get(label) is None else label_count_round.get(label))
                          for label in labels_order}

        total = sum(cleaned_counts.values())

        if total == 0:
            # 防止0除错误，如果没有任何label
            label_distribution = [0.0, 0.0, 0.0]
        else:
            label_distribution = [cleaned_counts[label] / total for label in labels_order]

        # 加入新的labels字段
        data["labels"] = label_distribution

        # 保存
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 完成！每行已新增 labels 分布，保存到：", output_path)


✅ 完成！每行已新增 labels 分布，保存到： /Users/phoebeeeee/ongoing/Beyond-noise/dataset/repeated/varierr_original/varierr_peer_no_error_dist.json


In [69]:
## clean no error

import json
from pathlib import Path
from tqdm import tqdm

def normalize_label_dist(chaos_dict):
    # 标签顺序固定：["entailment", "neutral", "contradiction"]
    label_order = ["entailment", "neutral", "contradiction"]
    values = [(chaos_dict.get(k) or 0.0) for k in label_order]  # 将 None 转换为 0.0
    total = sum(values)
    if total == 0:
        return [0.0] * 3
    return [v / total for v in values]


def process_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, desc=f"Processing {input_path}"):
            raw = json.loads(line)
            out = {
                "uid": raw.get("id", raw.get("uid")),
                "premise": raw.get("context") or raw["example"]["premise"],
                "hypothesis": raw.get("statement") or raw["example"]["hypothesis"],
                "label": raw.get("labels")
            }
            fout.write(json.dumps(out, ensure_ascii=False) + "\n")

# 示例调用方式
if __name__ == "__main__":
    input_path =  "/Users/phoebeeeee/ongoing/Beyond-noise/dataset/repeated/no_error/varierr_peer_no_error_dist.json"
    output_path ="/Users/phoebeeeee/ongoing/Beyond-noise/dataset/repeated/no_error/varierr_peer_no_error_cleaned.json"
    process_file(input_path, output_path)


Processing /Users/phoebeeeee/ongoing/Beyond-noise/dataset/repeated/no_error/varierr_peer_no_error_dist.json: 500it [00:00, 27545.18it/s]
